# Experiment rollout

In the previous pages, I had two variants, so I split users into to two groups.

It's often useful to split users (or whatever the experiment unit is!) into many groups and assign those to variants. Because users are randomly assigned to a group, the groups act like interchangeable representative cross-sections.

This can be used for a couple of things:
* Have different sized groups. Useful for rolling out an experiment in phases.
* Split traffic between two experiments, so a user is shown only one of the experiments.
* Limiting experimentation to a subset of users.

Heads up, as I start showing things to assignment groups, I'll start to change them. If I show one assignment group a bad variant, they might no longer be a representative cross-section. Salts can help a little here by shuffling and reassigning groups.

A lot of the more advanced experiment assignments depend on a combination of assignment groups.


## Example: Experiment rollout

So far, I've been showing experiments with a 50-50 split.

Now I have a situation where I don't want to show it to 50% of users right away. Instead, I want to start with 20% of users, and then move up to 50%. I also want to make it once someone starts seeing the new feature, I shouldn't take it away.

Here are the rules again.

* ① Given an experimental unit, return the value of the variant to show
* ② The same experimental unit is assigned to the same variant
* ③ Different experimental units are randomly assigned
* ④ The proportion of experimental units assigned to each variant is the proper proportion.

For ④, let's have the phase1 experiment show 20% of users blue, and phase 2 show 50%.

* Ⓐ Devices that see blue in phase 1 should continue to see it in phase 2.

| **Phase 1** |
|-------------|
| <img width=33% src="files/rollout-phase-1.png"> |
| **Phase 2** |
| <img width=33% src="files/rollout-phase-2.png"> |

### Disclaimers

This way of doing experiment rollout might not be the best idea in all cases.

I should make sure I have proper logging in place so I can tell which phase an assignment takes place.

Also, when I do analysis, that 20% of users who were shown blue in phase 1 are now a little different than the 30% who saw red in phase 1 and blue in phase 2. I might want to analyze the 3 groups separately.

In [9]:
import pandas as pd

from utils import spoilers
from utils.pretty import pp
from utils.simulate import n_different_users
from utils.simulate import same_user_n_times

In [10]:
# Phase 2: 80-20
def choose_color_assignment_phase_1(user_id):
    key = "{}|color".format(user_id)
    slot = spoilers.hash_func(key) % 20
    if slot < 16:
        return 'red'
    else:
        return 'blue'


# Phase 2: 50-50
def choose_color_assignment_phase_2(user_id):
    key = "{}|color".format(user_id)
    slot = spoilers.hash_func(key) % 20
    if slot < 10:
        return 'red'
    else:
        return 'blue'    

### ① Given a user_id, return the string of the color to show

In [11]:
# Don't mind me. This prints the user id, and what it was assigned to in phase 1 and phase 2.
def phase_info(user_id):
    print("user_id:{}\tphase 1: {}\tphase 2: {}".format(
        user_id, 
        choose_color_assignment_phase_1(user_id=user_id), 
        choose_color_assignment_phase_2(user_id=user_id)))


# Let's look at a few
phase_info(1) # Stays in red
phase_info(2) # Moves from red to blue
phase_info(4) # Stays in blue

user_id:1	phase 1: red	phase 2: red
user_id:2	phase 1: red	phase 2: blue
user_id:4	phase 1: blue	phase 2: blue


### ② The same user_id is assigned to the same color

In [12]:
pp(
    same_user_n_times(choose_color_assignment_phase_1, n=10, key='phase1')
)

,user_id,phase1
0,1,red
1,1,red
2,1,red
3,1,red
4,1,red
5,1,red
6,1,red
7,1,red
8,1,red
9,1,red


In [13]:
pp(
    same_user_n_times(choose_color_assignment_phase_2, n=10)
)

,user_id,color
0,1,red
1,1,red
2,1,red
3,1,red
4,1,red
5,1,red
6,1,red
7,1,red
8,1,red
9,1,red


### ③ Different user_ids are randomly assigned

In [14]:
pp(
    n_different_users(choose_color_assignment_phase_1, n=10, key='phase1')
)

,user_id,phase1
0,0,red
1,1,red
2,2,red
3,3,red
4,4,blue
5,5,red
6,6,red
7,7,red
8,8,blue
9,9,blue


In [15]:
pp(
    n_different_users(choose_color_assignment_phase_2, n=10, key='phase2')
)

,user_id,phase2
0,0,red
1,1,red
2,2,blue
3,3,red
4,4,blue
5,5,blue
6,6,blue
7,7,red
8,8,blue
9,9,blue


### ④ The proportion of user_ids that see red and blue is the correct proportion

In [16]:
n_different_users(choose_color_assignment_phase_1, n=10000, key='phase1').groupby('phase1').count()

,user_id
phase1,
blue,2003
red,7997


In [17]:
n_different_users(choose_color_assignment_phase_2, n=10000, key='phase2').groupby('phase2').count()

,user_id
phase2,
blue,4954
red,5046


### Ⓐ Devices that see blue in phase 1 should continue to see it in phase 2

This is like the usual graph, but shows phase1 and phase2 side-by-side.
Anyone who saw blue in *phase 1* should also see it in *phase 2*.

In [18]:
pp(
    pd.merge(
        n_different_users(choose_color_assignment_phase_1, n=100, key='phase1'), 
        n_different_users(choose_color_assignment_phase_2, n=100, key='phase2'), 
        on='user_id'
    )
)

,user_id,phase1,phase2
0,0,red,red
1,1,red,red
2,2,red,blue
3,3,red,red
4,4,blue,blue
5,5,red,blue
6,6,red,blue
7,7,red,red
8,8,blue,blue
9,9,blue,blue


To summarize this, I can count how many saw each color in the two phases.
I should watch out for phase 1 blue to phase 2 red, since that would break this requirement.

In [19]:
pd.merge(
        n_different_users(choose_color_assignment_phase_1, n=10000, key='phase1'), 
        n_different_users(choose_color_assignment_phase_2, n=10000, key='phase2'), 
        on='user_id'
    ).groupby(['phase1', 'phase2']).count()

user_id
phase1 phase2         
blue   blue       2003
red    blue       2951
       red        5046

# Summary

In this notebook, I extended it to use assignment groups, so it could support experiment rollout.

# Next

That's it! If you're interested in learning more about assignment, here are a few papers:

 - *Designing and Deploying Online Field Experiments* (Bakshy et al). Open sourced! http://facebook.github.io/planout/
 - *Controlled experiments on the web: survey and practical guide* (Kohavi et al)
 - *Overlapping Experiment Infrastructure: More, Better, Faster Experimentation* (Tang et al)
 - *From Infrastructure to Culture: A/B Testing Challenges in Large Scale Social Networks* (Xu et al)

# TOC
- **[0. Introduction](0.Introduction.ipynb)**: What a good `choose_color_assignment` function looks like.
- **[1. Experimental Units](1.ExperimentalUnits.ipynb)**: What happens when I don't pay attention to experimental units.
- **[2. Deterministic Assignment](2.DeterministicAssignment.ipynb)**: What it looks like to deterministically assign
- **[3. Scaling](3.Scaling.ipynb)**: How not to run two experiments at the same time.
- **[4. Rollout](4.Rollout.ipynb)**: How to gradually show users a new experiment.